In [1]:
%pip install wfdb
!git clone https://github.com/MIT-LCP/wfdb-python
%cd wfdb-python

from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import posixpath

import wfdb
from wfdb import processing

Note: you may need to restart the kernel to use updated packages.


Cloning into 'wfdb-python'...
Updating files:  71% (170/239)
Updating files:  72% (173/239)
Updating files:  73% (175/239)
Updating files:  74% (177/239)
Updating files:  75% (180/239)
Updating files:  76% (182/239)
Updating files:  77% (185/239)
Updating files:  78% (187/239)
Updating files:  79% (189/239)
Updating files:  80% (192/239)
Updating files:  81% (194/239)
Updating files:  82% (196/239)
Updating files:  83% (199/239)
Updating files:  84% (201/239)
Updating files:  85% (204/239)
Updating files:  86% (206/239)
Updating files:  87% (208/239)
Updating files:  88% (211/239)
Updating files:  89% (213/239)
Updating files:  90% (216/239)
Updating files:  91% (218/239)
Updating files:  92% (220/239)
Updating files:  93% (223/239)
Updating files:  94% (225/239)
Updating files:  95% (228/239)
Updating files:  96% (230/239)
Updating files:  97% (232/239)
Updating files:  98% (235/239)
Updating files:  99% (237/239)
Updating files: 100% (239/239)
Updating files: 100% (239/239), done.


c:\Users\jitbh\OneDrive\Documents\GitHub\FinalYearProject---BP-from-ECG-PPG\wfdb-python


In [ ]:
# dbs = wfdb.get_dbs()
# display(dbs)

ecg = wfdb.rdheader('a19', pn_dir='challenge-2010/set-a/')

if "ECG I" in ecg.sig_name and "ABP" in ecg.sig_name:
    print("Yes")
else:
    print("No")

In [6]:
ecg1abp = []
ecg2abp = [] 
ecg12abp = []

for i in range(0,100):

    if i < 10:
        fileNames = "c0{}".format(i)
    else:
        fileNames = "c{}".format(i)

    ecg = wfdb.rdheader(fileNames, pn_dir='challenge-2010/set-c/')
    if "I" in ecg.sig_name and "ABP" in ecg.sig_name:
        if "II" in ecg.sig_name:
            ecg12abp.append(fileNames)
        else:
            ecg1abp.append(fileNames)
    if "II" in ecg.sig_name and "ABP" in ecg.sig_name:
        if "I" in ecg.sig_name:
            ecg12abp.append(fileNames)
        else:
            ecg2abp.append(fileNames)

ecg12abp = list( dict.fromkeys(ecg12abp) )

print("ECG1", ecg1abp)
print("ECG1 size:", len(ecg1abp))
print("ECG2", ecg2abp)
print("ECG2 size:", len(ecg2abp))
print("ECG12", ecg12abp)
print("ECG12 size:", len(ecg12abp))


ECG1 []
ECG1 size: 0
ECG2 ['c00', 'c03', 'c04', 'c05', 'c07', 'c08', 'c09', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c19', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c32', 'c33', 'c34', 'c35', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c71', 'c72', 'c73', 'c74', 'c75', 'c76', 'c77', 'c78', 'c79', 'c80', 'c82', 'c83', 'c87', 'c88', 'c90', 'c91', 'c92', 'c93', 'c95', 'c96', 'c97', 'c98', 'c99']
ECG2 size: 81
ECG12 ['c02', 'c06', 'c18', 'c30', 'c36', 'c43', 'c62', 'c84', 'c85', 'c94']
ECG12 size: 10


In [10]:
ecg1PPGabp = []
ecg12PPGabp = []
ecg12abp = []
ecg2PPGabp = []
ecg1abp = []
ecg2abp = [] 


for i in range(0,100):

    if i < 10:
        fileNames = "c0{}".format(i)
    else:
        fileNames = "c{}".format(i)

    ecg = wfdb.rdheader(fileNames, pn_dir='challenge-2010/set-c/')

    if "II" in ecg.sig_name and "ABP" in ecg.sig_name:
        if "I" in ecg.sig_name:
            if "PLETH" in ecg.sig_name:
                ecg12PPGabp.append(fileNames)
            else:
                ecg12abp.append(fileNames)
        else:
            if "PLETH" in ecg.sig_name:
                ecg2PPGabp.append(fileNames)
            else:
                ecg2abp.append(fileNames)

    if "I" in ecg.sig_name and "ABP" in ecg.sig_name:
        if "II" in ecg.sig_name:
            if "PLETH" in ecg.sig_name:
                ecg12PPGabp.append(fileNames)
            else:
                ecg12abp.append(fileNames)
        else:
            if "PLETH" in ecg.sig_name:
                ecg1PPGabp.append(fileNames)
            else:
                ecg1abp.append(fileNames)    



ecg12PPGabp = list( dict.fromkeys(ecg12PPGabp))
ecg12abp = list( dict.fromkeys(ecg12abp) )

print("ECG1ABP", ecg1abp)
print("ECG1ABP size:", len(ecg1abp))
print("ECG1PPGABP", ecg1PPGabp)
print("ECG1PPGABP size:", len(ecg1PPGabp))
print("ECG12PPGABP", ecg12PPGabp)
print("ECG12PPGABP size:", len(ecg12PPGabp))
print("ECG2", ecg2abp)
print("ECG2 size:", len(ecg2abp))
print("ECG2PPGABP", ecg2PPGabp)
print("ECG2PPGABP size:", len(ecg2PPGabp))
print("ECG12", ecg12abp)
print("ECG12 size:", len(ecg12abp))


ECG1ABP []
ECG1ABP size: 0
ECG1PPGABP []
ECG1PPGABP size: 0
ECG12PPGABP ['c02', 'c06', 'c18', 'c30', 'c36', 'c43', 'c62', 'c84', 'c85', 'c94']
ECG12PPGABP size: 10
ECG2 []
ECG2 size: 0
ECG2PPGABP ['c00', 'c03', 'c04', 'c05', 'c07', 'c08', 'c09', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c19', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c32', 'c33', 'c34', 'c35', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c71', 'c72', 'c73', 'c74', 'c75', 'c76', 'c77', 'c78', 'c79', 'c80', 'c82', 'c83', 'c87', 'c88', 'c90', 'c91', 'c92', 'c93', 'c95', 'c96', 'c97', 'c98', 'c99']
ECG2PPGABP size: 81
ECG12 []
ECG12 size: 0


In [ ]:
# Read a WFDB record using the 'rdrecord' function into a wfdb.Record object.
# Plot the signals, and show the data. Read the files hosted on PhysioNet https://physionet.org/content/challenge-2010/1.0.0/
str = 'a00'
# sig, fields = wfdb.rdsamp(str, channels=[0,1], pn_dir='challenge-2010/set-a/')
sig, fields = wfdb.rdsamp(str, sampfrom=70000, sampto=75000, channels=[1,2,3,4,5], pn_dir='challenge-2010/set-a/')
# Call the gateway wrsamp function, manually inserting fields as function input parameters
fmt = ['16', '16', '16', '16', '16']
units = ['mV', 'mV', 'mV', '?', 'mmHg']
names = ['ECGII', 'ECGV', 'ECGAVR', 'PLETH', 'ABP']
wfdb.wrsamp('ecg-record', fs=125, units=units, sig_name=names, p_signal=sig, fmt=fmt)
# The new file can be read
record = wfdb.rdrecord('ecg-record')
wfdb.plot_wfdb(record=record, plot_sym='true', title='Record ' +  str + ' from PhysioNet Challenge 2010', time_units='seconds') 
display(record.__dict__)
# Convert to seconds
# Snapshot for seconds

In [ ]:
# Read a WFDB record using the 'rdrecord' function into a wfdb.Record object.
# Plot the signals, and show the data. Read the files hosted on PhysioNet https://physionet.org/content/challenge-2010/1.0.0/
str = 'a01'
# sig, fields = wfdb.rdsamp(str, channels=[0,1], pn_dir='challenge-2010/set-a/')
sig, fields = wfdb.rdsamp(str, sampfrom=70000, sampto=75000, channels=[0,2,3,4,5], pn_dir='challenge-2010/set-a/')
# Call the gateway wrsamp function, manually inserting fields as function input parameters
fmt = ['16', '16', '16', '16', '16']
units = ['mmHg', '?', 'mV', 'mV', 'mV' ]
names = ['ABP', 'PLETH', 'ECGII', 'ECGV', 'ECGAVR']
wfdb.wrsamp('ecg-record', fs=125, units=units, sig_name=names, p_signal=sig, fmt=fmt)
# The new file can be read
record = wfdb.rdrecord('ecg-record')
wfdb.plot_wfdb(record=record, plot_sym='true', title='Record ' +  str + ' from PhysioNet Challenge 2010', time_units='seconds') 
display(record.__dict__)

In [ ]:
str = 'a02'
# sig, fields = wfdb.rdsamp(str, channels=[0,1], pn_dir='challenge-2010/set-a/')
sig, fields = wfdb.rdsamp(str, sampfrom=70000, sampto=75000, channels=[1,2,3,4,5], pn_dir='challenge-2010/set-a/')
# Call the gateway wrsamp function, manually inserting fields as function input parameters
fmt = ['16', '16', '16', '16', '16']
units = ['?', 'mV', 'mV', 'mV', 'mmHg']
names = ['PLETH', 'ECGII', 'ECGV', 'ECGAVR', 'CVP']
wfdb.wrsamp('ecg-record', fs=125, units=units, sig_name=names, p_signal=sig, fmt=fmt)
# The new file can be read
record = wfdb.rdrecord('ecg-record')
wfdb.plot_wfdb(record=record, plot_sym='true', title='Record ' +  str + ' from PhysioNet Challenge 2010', time_units='seconds') 
display(record.__dict__)
# Convert to seconds
# Snapshot for seconds


# go through all 100 training in Set A, note down what signals they come with. ECGII and ABP


# ECGV is redundant

In [ ]:
str = 'a03'
# sig, fields = wfdb.rdsamp(str, channels=[0,1], pn_dir='challenge-2010/set-a/')
sig, fields = wfdb.rdsamp(str, sampfrom=70000, sampto=75000, channels=[1,2,3,4,5], pn_dir='challenge-2010/set-a/')
# Call the gateway wrsamp function, manually inserting fields as function input parameters
fmt = ['16', '16', '16', '16', '16']
units = ['mV', 'mV', 'mV', '?', 'mmHg']
names = ['ECGII', 'ECGV', 'ECGAVR', 'PLETH', 'ABP']
wfdb.wrsamp('ecg-record', fs=125, units=units, sig_name=names, p_signal=sig, fmt=fmt)
# The new file can be read
record = wfdb.rdrecord('ecg-record')
wfdb.plot_wfdb(record=record, plot_sym='true', title='Record ' +  str + ' from PhysioNet Challenge 2010', time_units='seconds') 
display(record.__dict__)
# Convert to seconds
# Snapshot for seconds

In [ ]:
record = wfdb.rdheader('a10', pn_dir='challenge-2010/set-a/')
display(record.__dict__)

In [ ]:
str = 'a03'
# sig, fields = wfdb.rdsamp(str, channels=[0,1], pn_dir='challenge-2010/set-a/')
sig, fields = wfdb.rdsamp(str, sampfrom=70000, sampto=75000, channels=[1,2,3,4,5], pn_dir='challenge-2010/set-a/')
# Call the gateway wrsamp function, manually inserting fields as function input parameters
fmt = ['16', '16', '16', '16', '16']
units = ['mV', 'mV', 'mV', '?', 'mmHg']
names = ['ECGII', 'ECGV', 'ECGAVR', 'PLETH', 'ABP']
wfdb.wrsamp('ecg-record', fs=125, units=units, sig_name=names, p_signal=sig, fmt=fmt)
# The new file can be read
record = wfdb.rdrecord('ecg-record')
wfdb.plot_wfdb(record=record, plot_sym='true', title='Record ' +  str + ' from PhysioNet Challenge 2010', time_units='seconds') 
display(record.__dict__)
# Convert to seconds
# Snapshot for seconds

In [ ]:
# Write a WFDB record without using a Record object via the gateway wrsamp function.
# This is the basic way to write physical signals to a WFDB file. 

str = 'a00'
# Read part of a record from Physionet
sig, fields = wfdb.rdsamp('a00', sampfrom=0, sampto=250, channels=[1,3,4,5], pn_dir='challenge-2010/set-a/')


# Call the gateway wrsamp function, manually inserting fields as function input parameters
wfdb.wrsamp('ecg-record', fs=125, units=['mV', 'mV', '?', 'mmHg'], sig_name=['ECGI', 'ECGII', 'PLETH', 'ABP'], p_signal=sig, fmt=['16', '16', '16', '16'])


# wfdb.plot_wfdb(record=record, plot_sym='true', title='Record ' +  str + ' from PhysioNet Challenge 2010', time_units='seconds') 

# The new file can be read
record = wfdb.rdrecord('ecg-record')
wfdb.plot_wfdb(record=record, plot_sym='true', title='Record ' +  str + ' from PhysioNet Challenge 2010', time_units='seconds') 
display(record.__dict__)

# plt.plot(record.p_signal, )


#wfdb.plot_wfdb(record=record, title='Record a10 from PhysioNet Challenge 2010') 
#display(record.__dict__)

# ECG processing

In [ ]:
# Demo 19 - Use the GQRS detection algorithm and correct the peaks

def peaks_hr(sig, peak_inds, fs, title, figsize=(20, 10), saveto=None):
    "Plot a signal with its peaks and heart rate"
    # Calculate heart rate
    hrs = processing.hr.compute_hr(sig_len=sig.shape[0], qrs_inds=peak_inds, fs=fs)
    
    N = sig.shape[0]
    
    fig, ax_left = plt.subplots(figsize=figsize)
    ax_right = ax_left.twinx()
    
    ax_left.plot(sig, color='#3979f0', label='Signal')
    ax_left.plot(peak_inds, sig[peak_inds], 'rx', marker='x', 
                 color='#8b0000', label='Peak', markersize=12)
    ax_right.plot(np.arange(N), hrs, label='Heart rate', color='m', linewidth=2)

    ax_left.set_title(title)

    ax_left.set_xlabel('Time (ms)')
    ax_left.set_ylabel('ECG (mV)', color='#3979f0')
    ax_right.set_ylabel('Heart rate (bpm)', color='m')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax_left.tick_params('y', colors='#3979f0')
    ax_right.tick_params('y', colors='m')
    if saveto is not None:
        plt.savefig(saveto, dpi=600)
    plt.show()

# Load the WFDB record and the physical samples
record = wfdb.rdrecord('a10', pn_dir='challenge-2010/set-a/', sampfrom=0, sampto=10000, channels=[0])

record = wfdb.rdrecord('sample-data/100', sampfrom=0, sampto=10000, channels=[0])


# Use the GQRS algorithm to detect QRS locations in the first channel
qrs_inds = processing.qrs.gqrs_detect(sig=record.p_signal[:,0], fs=record.fs)

# Plot results
peaks_hr(sig=record.p_signal, peak_inds=qrs_inds, fs=record.fs,
         title="GQRS peak detection on record a10")
    
# Correct the peaks shifting them to local maxima
min_bpm = 20
max_bpm = 230
#min_gap = record.fs * 60 / min_bpm
# Use the maximum possible bpm as the search radius
search_radius = int(record.fs * 60 / max_bpm)
corrected_peak_inds = processing.peaks.correct_peaks(record.p_signal[:,0], 
                                                     peak_inds=qrs_inds,
                                                     search_radius=search_radius, 
                                                     smooth_window_size=150)

# Display results
print('Corrected GQRS detected peak indices:', sorted(corrected_peak_inds))
peaks_hr(sig=record.p_signal, peak_inds=sorted(corrected_peak_inds), fs=record.fs,
         title="Corrected GQRS peak detection on sampledata/100")